<a href="https://colab.research.google.com/github/Moses-Otu/DataEngineering/blob/main/spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import datetime as dt

In [ ]:
df1=pd.read_csv('/content/spotify_data_dictionary.csv')
df2=pd.read_csv('/content/spotify_history.csv')

In [ ]:
df1.shape

In [ ]:
#Understing the imported data
df1

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
#rename columns
df2.rename(columns={'ts':'timestamp','track_name':'track','artist_name':'artist','album_name':'album','reason_start':'start reason',
                    'reason_end': 'end reason','ms_played':'most played min','spotify_track_uri':'track url'}, inplace=True)
df2.columns

In [ ]:
#converting milliseconds to minuites
df2['most played min']=(df2['most played min']/60000).round(2)

In [ ]:
#cleaning tracks column
# Apply regex to remove ", /, -, #, and ' from the 'track' column
df2['track'] = df2['track'].astype(str).apply(lambda x: re.sub(r'["/#\'-]', '', x))


In [ ]:
#cleaning album column
# Apply regex to remove ", /, -, #, and ' from the 'track' column
df2['album'] = df2['album'].astype(str).apply(lambda x: re.sub(r'["/#\'-]', '', x))

In [ ]:
#casting the datetime column as type datetime
df2['timestamp']=pd.to_datetime(df2['timestamp'])

In [ ]:
df2.info()

In [ ]:
df2.describe()

In [ ]:
df2.isnull().sum()

In [ ]:
((df2.isnull().sum()/len(df2))*100).round(2)

In [ ]:
#dealing with nulls
df2['start reason']=df2['start reason'].fillna('unknown')
df2['end reason']=df2['end reason'].fillna('unknown')
df2.isnull().sum()

### **EXPLORATORY DATA ANALYSIS**

In [ ]:
#who are the most listened to artist
df2.groupby('artist')['most played min'].sum().sort_values(ascending=False).reset_index().head(10)

In [ ]:
#what are the most listened to songs
df2.groupby('track')['most played min'].sum().sort_values(ascending=False).reset_index().head(10)

In [ ]:
#comparing most played artist and track
df2.groupby(['track','artist'])['most played min'].sum().sort_values(ascending=False).reset_index().head(6)

Analysis above shows that most listened to artists may not neccessarily mean they have the most streamed tracks

In [ ]:
df2_fil = df2[['artist','track','most played min','album']][df2['artist'].isin(["The Beatles"])]
df2_fil

Calculating the percentage contribution of an artist's top track to their total streams. This will show if their success is due to a single track or multiple popular tracks:

In [ ]:
artist_total=df2.groupby('artist')['most played min'].sum().sort_values(ascending=False).reset_index()
track_contrib=df2.groupby(['artist','track'])['most played min'].sum().sort_values(ascending=False).reset_index()

merged = track_contrib.merge(artist_total, on='artist',suffixes=('_track', '_artist'))
merged


In [ ]:
merged['track_contribution'] = (merged['most played min_track'] / merged['most played min_artist']) * 100

#filtered = merged[merged['artist'].isin(['The Strokes'])]
merged.head(10)

In [ ]:
#Artists with one hit singles --artists with contribution >80%
onehitsingle = merged.sort_values('track_contribution',ascending=False)
onehitsingle

In [ ]:
#most used spotify streaming platform
mostusedplatform=df2.groupby('platform')['most played min'].sum().sort_values(ascending=False).reset_index()
mostusedplatform

In [ ]:
# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(mostusedplatform['platform'], mostusedplatform['most played min'], color='skyblue')

# Add labels and title
plt.xlabel('Platform', fontsize=12)
plt.ylabel('Total Plays', fontsize=12)
plt.title('Most Used Platforms by Total Plays', fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()

# Show the chart
plt.show()

In [ ]:
df2['album'].nunique()

In [ ]:
#most played albums
mostplayedalbum=df2.groupby(['album','artist'])['most played min'].sum().sort_values(ascending=False).reset_index().head(10)
mostplayedalbum

In [ ]:
#What are the most listened to tracks on the album
# Group by album, track, and artist, then sum the plays
pp = df2.groupby(['album', 'track', 'artist'])['most played min'].sum().reset_index()

# Filter for "The Beatles"
#pp = pp[pp['artist'] == 'The Beatles']

# Sort by track in ascending order
#pp = pp.sort_values('track', ascending=True)
pp = pp.sort_values(['album', 'most played min'], ascending=[True, False])

# Display the top 10 popular tracks for The Beatles
pp.head(10)


Feature Engineering to generate more insight

In [ ]:
df2['Dayoftheweek']=df2['timestamp'].dt.day_name()

In [ ]:
#analysis on weekday and streaming
day_wise_plays = df2.groupby('Dayoftheweek')['most played min'].sum().sort_values(ascending=False).reset_index()

In [ ]:
#order day of the week
day_wise_plays['Dayoftheweek'] = pd.Categorical(day_wise_plays['Dayoftheweek'], categories=[
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)

# Sort and reset index
day_wise_plays = day_wise_plays.sort_values('Dayoftheweek').reset_index(drop=True)

day_wise_plays

In [ ]:
#linechart

plt.figure(figsize=(10, 6))
sns.lineplot(x='Dayoftheweek', y='most played min', data=day_wise_plays.groupby('Dayoftheweek')['most played min'].sum().sort_values(ascending=False).reset_index())
plt.xlabel('Day of the week')
plt.ylabel('Total minutes played')
plt.title('Total minutes played per day of the week')
plt.show()


In [ ]:
#analysis on streaming per month
df2['month']=df2['timestamp'].dt.month_name()

In [ ]:
#analysis on weekday and streaming
month_wise_plays = df2.groupby('month')['most played min'].sum().reset_index()

month_wise_plays['month'] = pd.Categorical(month_wise_plays['month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)
#sort
month_wise_plays = month_wise_plays.sort_values('month').reset_index(drop=True)
month_wise_plays


In [ ]:
#linechart

plt.figure(figsize=(10, 6))
sns.lineplot(x='month', y='most played min', data=month_wise_plays.groupby('month')['most played min'].sum().sort_values(ascending=False).reset_index())
plt.xlabel('Day of the week')
plt.ylabel('Total minutes played')
plt.title('Total minutes played per month of the year')
plt.show()


In [ ]:
#what time of the day has more streaming?
df2['hour']=df2['timestamp'].dt.hour
#assigning time of day
def time_of_day(hour):
    if 6 >= hour < 12:
        return 'Morning'
    elif 12 >= hour < 17:
        return 'Afternoon'
    elif 17 >= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to the 'hour' column to create a new 'time_of_day' column
df2['time_of_day'] = df2['hour'].apply(time_of_day)
df2.groupby('time_of_day')['most played min'].sum().sort_values(ascending=False).reset_index()

In [ ]:
#what time of day do we have more streams in october
october_df=df2[df2['month']=='October']
oct_fill=october_df.groupby('month')['time_of_day'].value_counts().reset_index()
oct_fill

In [ ]:
# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(oct_fill['time_of_day'], oct_fill['count'], color='pink')

# Add labels and title
plt.xlabel('time_of_day', fontsize=12)
plt.ylabel('count', fontsize=12)
plt.title('Most Used Platforms by Total Plays', fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()

# Show the chart
plt.show()